<a href="https://colab.research.google.com/github/anderson-vasconcelos/data_enginner_getnet/blob/main/Desafio_Engenheiro_Getnet_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [11]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

transacoes = [{'trasaction_id': 1, 'client_id': 3545, 'total_amount': 3000, 'discount_percentage': 6.99},
              {'trasaction_id': 2, 'client_id': 3545, 'total_amount': 4500, 'discount_percentage': 0.45},
              {'trasaction_id': 3, 'client_id': 3509, 'total_amount': 69998, 'discount_percentage': 0.00},
              {'trasaction_id': 4, 'client_id': 3510, 'total_amount': 1, 'discount_percentage': None},
              {'trasaction_id': 5, 'client_id': 4510, 'total_amount': 34, 'discount_percentage': 40.00}]
contratos = [{'contratct_id': 3, 'client_id': 3545, 'client_name': 'Magazine Luana', 'percentage': 2.00, 'is_active': True},
             {'contratct_id': 4, 'client_id': 3545, 'client_name': 'Magazine Luana', 'percentage': 1.95, 'is_active': False},
             {'contratct_id': 5, 'client_id': 3509, 'client_name': 'Lojas Italianas', 'percentage': 1.00, 'is_active': True},
             {'contratct_id': 6, 'client_id': 3510, 'client_name': 'Carrerfive', 'percentage': 3.00, 'is_active': True}]

sc = SparkSession.builder.master('local[*]').getOrCreate()

df_transacoes = sc.createDataFrame(transacoes)
df_contratos = sc.createDataFrame(contratos)

df_transacoes.fillna(0,["discount_percentage"]) \
  .join(df_contratos, df_transacoes["client_id"] ==  df_contratos["client_id"], "inner") \
  .filter(F.col('is_active') == True) \
  .withColumn('total_amount_percentage', F.col('total_amount') * (F.col('percentage') / 100.00)) \
  .withColumn('total_amount_percentage_discount_percentage', F.col("total_amount_percentage") * (F.col('discount_percentage') / 100.00)) \
  .withColumn('total_gain', F.col('total_amount_percentage') - F.col('total_amount_percentage_discount_percentage')) \
  .agg(F.sum("total_gain")) \
  .select(F.round(F.col("sum(total_gain)"),3).alias("total_gain")) \
  .show()

spark-2.4.4-bin-hadoop2.7/python/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+----------+
|total_gain|
+----------+
|   845.411|
+----------+

